In [48]:
!pip install numpy pandas scikit-learn torch

In [49]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [50]:
df=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [51]:
df.drop("customerID",axis=1,inplace=True)
df["TotalCharges"]=pd.to_numeric(df["TotalCharges"],errors="coerce") # converting str->int and if error then to nan
df.dropna(inplace=True)

In [52]:
df=pd.get_dummies(df,drop_first=True) # one hot enconding

In [53]:
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,0,1,29.85,29.85,False,True,False,False,True,False,...,False,False,False,False,False,True,False,True,False,False
1,0,34,56.95,1889.50,True,False,False,True,False,False,...,False,False,False,True,False,False,False,False,True,False
2,0,2,53.85,108.15,True,False,False,True,False,False,...,False,False,False,False,False,True,False,False,True,True
3,0,45,42.30,1840.75,True,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
4,0,2,70.70,151.65,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,False,True


In [54]:
X=df.drop("Churn_Yes",axis=1).values
y=df["Churn_Yes"].values

In [55]:
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [56]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=42)

In [57]:
Xtrain=torch.tensor(Xtrain,dtype=torch.float32)  # converting numpy arr to pytorch multi dimensional arr
ytrain=torch.tensor(ytrain.reshape(-1,1),dtype=torch.float32)  # reshapes convert each col to a vector from [1,2,3] => [[1],[2],[3]]
Xtest=torch.tensor(Xtest,dtype=torch.float32)
ytest=torch.tensor(ytest.reshape(-1,1),dtype=torch.float32)

In [58]:
class churnNet(nn.Module): # inherting
    def __init__(self,inputDim):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(inputDim,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,1),
            nn.Sigmoid()
        )

    def forward(self,x):
        return self.net(x)

In [59]:
model=churnNet(Xtrain.shape[1]) # makes a model with num of col=dimensions
lossFn=nn.BCELoss() # since our output is binary  so we use bce
opt=torch.optim.Adam(model.parameters(),lr=0.001) # optimizer

In [60]:
for epoch in range(250):
    pred=model(Xtrain) # making pred on training data
    loss=lossFn(pred,ytrain) # finding loss
    opt.zero_grad() # removing previous gradient
    loss.backward() # back prop
    opt.step() # updating based on learning rate

with torch.no_grad():
    pred=(model(Xtest)>0.5).float()
    acc=(pred==ytest).float().mean()

In [61]:
print("accuracy:",acc.item())

accuracy: 0.7903340458869934
